<a href="https://colab.research.google.com/github/Karasiari/Graphs/blob/main/Graphs_intial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import os # Выберите, откуда тяните папку
from google.colab import files

# По папке формируем словарь графов со структурой -
#                                                {'название графа (как в папке)':
#                                                       {'adj_matrix': матрица смежности,
#                                                        'traffic_matrix': матрица корреспонденций,
#                                                        'nodes_params': {'degrees': список степеней вершин,
#                                                                         'source_demands': список суммарного входящего траффика,
#                                                                         'target_demands': список суммарного исходящего траффика,
#                                                                         'sum_demands': список усредненного траффика (source + target) / 2
#                                                                        }
#                                                       }
#                                                }

def csv_to_graph(path, demands_path):
  Df = pd.read_csv(path, header=None, names = ['id', 'source', 'target', 'length'])
  Traffic = pd.read_csv(demands_path, header=None, names = ['id', 'source', 'target', 'bitrate'])
  df = Df.iloc[1:].copy()
  traffic = Traffic.iloc[1:].copy()
  df['source'] = df['source'].astype(int)
  df['target'] = df['target'].astype(int)
  df['length'] = df['length'].astype(float)
  traffic['source'] = traffic['source'].astype(int)
  traffic['target'] = traffic['target'].astype(int)
  traffic['bitrate'] = traffic['bitrate'].astype(int) // 100
  traffic_grouped = traffic.groupby(['source', 'target']).sum().reset_index()

  sources = df['source'].tolist()
  targets = df['target'].tolist()
  lengths = df['length'].tolist()

  unique_vertices = set()
  for source, target in zip(sources, targets):
    unique_vertices.add(source)
    unique_vertices.add(target)
  vertex_mapping = {old: new for new, old in enumerate(unique_vertices)}
  num_vertices = len(unique_vertices)

  adj_matrix = np.zeros((num_vertices, num_vertices))
  traffic_matrix = np.zeros((num_vertices, num_vertices))

  for source, target, length in zip(sources, targets, lengths):
    new_source = vertex_mapping[source]
    new_target = vertex_mapping[target]
    adj_matrix[new_source, new_target] = length
    adj_matrix[new_target, new_source] = length

  traffic_grouped['source'] = traffic_grouped['source'].map(vertex_mapping)
  traffic_grouped['target'] = traffic_grouped['target'].map(vertex_mapping)
  for _, row in traffic_grouped.iterrows():
    traffic_matrix[row['source'], row['target']] = row['bitrate']

  nodes_params = {}

  degrees = list(np.count_nonzero(adj_matrix, axis=1).astype(np.float64))
  nodes_params['degrees'] = degrees

  source_demands = list(np.sum(traffic_matrix, axis=1))
  nodes_params['source_demands'] = source_demands

  target_demands = list(np.sum(traffic_matrix, axis=0))
  nodes_params['target_demands'] = target_demands

  sum_demands = [(i + j)/2 for (i,j) in zip(source_demands, target_demands)]
  nodes_params['sum_demands'] = sum_demands

  return (adj_matrix, traffic_matrix, nodes_params)

def get_graphs(base_path):
  Graphs = {}
  csv_tables = []
  for folder_name in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder_name)
    csv_path = os.path.join(folder_path, 'links.csv')
    csv_path_demands = os.path.join(folder_path, 'demands.csv')
    csv_tables.append(folder_name)

    (adj_matrix, traffic_matrix, nodes_params) = csv_to_graph(csv_path, csv_path_demands)
    Graphs[folder_name] = {'adj_matrix': adj_matrix, 'traffic_matrix': traffic_matrix, 'nodes_params': nodes_params}

  return Graphs

path_to_folder = '/content/drive/MyDrive/Кола' # Свой путь
Graphs = get_graphs(path_to_folder)

In [6]:
Graphs['cola_test60'] # Выводим пример

{'adj_matrix': array([[ 0.,  0., 60., 60., 60.],
        [ 0.,  0., 60., 60., 60.],
        [60., 60.,  0., 60.,  0.],
        [60., 60., 60.,  0., 60.],
        [60., 60.,  0., 60.,  0.]]),
 'traffic_matrix': array([[ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [20.,  0.,  0.,  0.,  0.]]),
 'nodes_params': {'degrees': [np.float64(3.0),
   np.float64(3.0),
   np.float64(3.0),
   np.float64(4.0),
   np.float64(3.0)],
  'source_demands': [np.float64(20.0),
   np.float64(0.0),
   np.float64(20.0),
   np.float64(0.0),
   np.float64(20.0)],
  'target_demands': [np.float64(20.0),
   np.float64(40.0),
   np.float64(0.0),
   np.float64(0.0),
   np.float64(0.0)],
  'sum_demands': [np.float64(20.0),
   np.float64(20.0),
   np.float64(10.0),
   np.float64(0.0),
   np.float64(10.0)]}}